In [1]:
import folium, json


In [2]:
with open("../geojson/municipi.geojson") as f:
    stazioni_geo=json.load(f)
    
for item in stazioni_geo["features"]:
    item["municipio"]=item['id']

In [3]:
import pandas as pd
aria_stazioni = pd.read_csv("../csv/qaria_stazione.csv")
aria_stazioni

,id_amat,nome,id_arpa,inizio_operativita,fine_operativita,inquinanti,LONG_X_4326,LAT_Y_4326,Location
0,1,p.le Abbiategrasso,126,1900/01/01,2017/08/31,NO2,9.18219,45.432301,"(45.432300567627, 9.18218994140625)"
1,3,viale Liguria,107,1900/01/01,2099/12/31,"CO_8h, NO2",9.16944,45.444199,"(45.4441986083984, 9.16944026947021)"
2,4,viale Marche,2,1900/01/01,2099/12/31,"NO2, PM10, CO_8h, C6H6",9.19084,45.496300,"(45.4962997436523, 9.19083976745605)"
3,5,Parco Lambro,124,1900/01/01,2017/08/31,"O3, NO2",9.24730,45.499599,"(45.4995994567871, 9.24730014801025)"
4,7,Verziere,85,1900/01/01,2099/12/31,"PM10, O3, NO2, CO_8h",9.19534,45.463501,"(45.4635009765625, 9.19534015655518)"
5,8,p.le Zavattari,7,1900/01/01,2017/08/31,"CO_8h, C6H6, NO2",9.14177,45.476101,"(45.4761009216309, 9.141770362854)"
6,9,via Juvara,7,1900/01/01,2007/06/11,"PM10, NO2, PM25, SO2, O3",9.22045,45.473499,"(45.4734992980957, 9.22045040130615)"
7,2,via Pascal,100,1900/01/01,2099/12/31,"C6H6, PM10, NO2, SO2, SO2, PM25, O3",9.23478,45.474098,"(45.4740982055664, 9.23478031158447)"
8,6,via Senato,125,1900/01/01,2099/12/31,"PM25, C6H6, CO_8h, PM10, NO2",9.19792,45.470501,"(45.4705009460449, 9.19791984558105)"


In [4]:
mega = pd.read_csv("../csv/mega_csv_NO2.csv")
mega

,stazione_id,data,inquinante,valore,nome,posizione
0,1,2007-01-01,NO2,89.0,p.le Abbiategrasso,"(45.432300567627, 9.18218994140625)"
1,4,2007-01-01,NO2,87.0,viale Marche,"(45.4962997436523, 9.19083976745605)"
2,8,2007-01-01,NO2,76.0,p.le Zavattari,"(45.4761009216309, 9.141770362854)"
3,7,2007-01-01,NO2,81.0,Verziere,"(45.4635009765625, 9.19534015655518)"
4,9,2007-01-01,NO2,79.0,via Juvara,"(45.4734992980957, 9.22045040130615)"
...,...,...,...,...,...,...
30197,6,2018-12-28,NO2,68.0,via Senato,"(45.4705009460449, 9.19791984558105)"
30198,8,2018-12-28,NO2,NaN,p.le Zavattari,"(45.4761009216309, 9.141770362854)"
30199,1,2018-12-28,NO2,NaN,p.le Abbiategrasso,"(45.432300567627, 9.18218994140625)"
30200,2,2018-12-28,NO2,70.0,via Pascal,"(45.4740982055664, 9.23478031158447)"


In [5]:
dff = mega.groupby(by=['stazione_id'])
dfff = dff["valore"].mean()    

In [6]:
#prova=mega.groupby('stazione_id')
#prova.head()

In [7]:
#per municipio
zero = 60
per_zone_pollution = pd.DataFrame(data={
    'data': [(dfff[7]+dfff[6])/2,zero,dfff[4],zero,(dfff[5]+dfff[2]+dfff[9])/3,zero,dfff[8],dfff[3],dfff[9]],
    'index':[1,2,3,4,5,6,7,8,9]})

In [8]:
mappa=folium.Map(location =[45.47,9.2], zoom_start=12, tiles="CartoDB positron")
mappa.save("../test/maps/map.html")

#marker
for _, i in aria_stazioni.iterrows():
    folium.Marker(location=[i['LAT_Y_4326'], i['LONG_X_4326']], popup=str(i['id_amat']), tooltip= "Clicca per info").add_to(mappa)

#folium.CircleMarker(location= [45.4654219,9.1859243]).add_to(map)
folium.Choropleth(geo_data=stazioni_geo,
                 data=per_zone_pollution,
                 columns=["index", "data"],
                 key_on="feature.municipio",#serve per connettere geojson e dataframe
                 bins=6,
                 fill_color="YlOrRd", #da giallo a verde
                 nan_fill_opacity=None,
                 line_color='black',
                 line_weight=1,
                 line_opacity=1,
                 name=None,
                 legend_name="Qualità aria municipi milano",
                 overlay=True,
                 control=True,
                 show=True,
                 topojson=None,
                 smooth_factor=None,
                 highlight=None).add_to(mappa)

mappa